In [2]:
import pandas as pd
import numpy as np
import seaborn as sn
import total_aaindex_parser
import joblib

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import hyperparameter_tuning
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectPercentile, f_classif

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier


In [ ]:
#Here the Training data is Read in and converted to numbers    
f = open('data/project_training.txt', "r")
inputFile = open('data/test_input.txt', "r")
outputFile = open('data/test_output.txt', 'w')

In [3]:
def parseAaIndex():
    source = "data/aaindex1.txt"
    if not os.path.isfile(source):
        source = "../data/aaindex1.txt"
    input_file = open(source, "r")
    lines = input_file.read().splitlines()
    totalaaIndex = []
    counter = 0
    for i in range(len(lines)):
        if lines[i][0] == "I":
            bothLines = lines[i + 1] + lines[i + 2]
            partielaaIndex = bothLines.split()
            totalaaIndex.append(partielaaIndex)
            counter += 1
    input_file.close()
    return totalaaIndex


# Entfernt Eigenschaften in denen NA vorkommt und Floatet die Liste
# Out: neue Liste der aaIndex-Eigenschaftens
def getFeaturesforAAX():
    a = parseAaIndex()
    rmv = []
    for feature in a:
        if 'NA' in feature:
            rmv.append(feature)
    for feature in rmv:
        a.remove(feature)
    lst = []
    for feature in a:
        #b = feature.apply(float)
        b = list(map(float, feature))
        lst.append(b)
    return lst


def parseProjectTraining():
    training_data = []
    source = "data/project_training.txt"
    if not os.path.isfile(source):
        source = "../data/project_training.txt"
    input_file = open(source, "r")
    ligands = []
    labels_str = []
    next(input_file)
    for line in input_file:
        ligands.append(line.split(None, 1)[0])
        labels_str.append(line[-2])
    print(ligands, labels_str)
    input_file.close()
    labels = []
    for i in range(len(labels_str)):
        if(labels_str[i]!='\t'):
            temp = int(labels_str[i])
            if temp:
                labels.append(temp)
            else:
                labels.append(0)
    for x in range(0, len(ligands)):
        if(line!='\t'):
            training_data.append([ligands[x], labels[x]])

    return training_data, ligands, labels

# Gibt Trainingsdaten aus
# Out: Trainingsdaten
def getTrainingData():
    return trainingData


# Gibt Liganden der Trainingsdaten aus
# Out: Liganden des Trainingsdatensatzes
def getLigands():
    return ligands


# Gibt Labels der Trainingsdaten aus
# Out: Labels des Trainingsdatensatzes
def getLabels():
    return labels


# Gibt Liste mit Features für die Trainingsdaten aus
# Out: Features für Aminosäuren
def getFeaturesForAS(x):
    features_for_as = []
    for feature in getFeaturesforAAX():
        features_for_as.append(feature[x])
    return features_for_as


# Weist jeder Aminosäure seine Features zu
# Out: Dictionary das jeder Aminosäure seine Eigenschaften zuweist
def setAllFeatures():
    aas = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]
    feature_dic = {}
    for x in range(0, len(aas)):
        feature_dic.update({aas[x]: getFeaturesForAS(x)})
    return feature_dic


# Stellt Peptide als Feature-Vektor dar
# In: Liganden und Features
# Out: Liganden als Feature-Vektor
def featureLigands(ligands, features):
    a = setAllFeatures()
    number_of_features = len(a["A"])
    ligands_featured = []
    for x in range(0, len(ligands)):
        peptide = []
        for y in range(0, number_of_features):
            for char in ligands[x]:
                peptide.append(features[char][y])
        ligands_featured.append(peptide)
    return ligands_featured


# Bereitet Trainingsdaten für SVM auf
# Out: Skalierte und Selektierte Features und Labels der Trainingsdaten
def prepareTrainingData():
    # Einlesen der Trainingsdaten und casten in Numpy Array
    labels = np.array(getLabels())
    features = np.array(featureLigands(getLigands(), setAllFeatures()))
    # Skalierung der Trainingsdaten mit MinMaxScaler
    scaler = preprocessing.MinMaxScaler()
    scaled_features = scaler.fit_transform(features)
    # Feature Selection mit 10 Percentil
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(scaled_features, labels)
    # Speichere Selektor um ihn bei INPUT später auch zu verwenden
    joblib.dump(selector, './selector/selector.pkl')
    selected_features = selector.transform(scaled_features)
    return selected_features, labels

# Lädt Trainingsdaten von training_data.txt
trainingData, ligands, labels = parseProjectTraining()
features, labels = prepareTrainingData()

['NFFHASLAY', 'YQKKNASVY', 'AAFLDDNAF', 'IQNKLSSTF', 'KLAEIFQPF', 'FAAAAARTL', 'MQQSGDEAF', 'KLRKKSSFY', 'GLGGDASAY', 'LQKVPHTRY', 'IQAGVDRFY', 'FQVNRFTGY', 'YFRNSGMTY', 'TVSPSAPTY', 'KTIQGGLGW', 'VMLDWGIEL', 'KLEGKIVQY', 'RMGAAVTPY', 'SQHNYRPGY', 'RGRAATMAL', 'QLFIKDYRY', 'FQLYSDLAH', 'FVFEATKLY', 'SQLEMCEKY', 'KVAAAGVSY', 'YVRGYLRGY', 'MQLQLNCAY', 'LLKPGGVQW', 'YLCEDTITY', 'VAGGTSSVY', 'WQFGPSTYY', 'ALEPGFKDY', 'IQGTLAKAY', 'TTFPVNGGY', 'ILSPHNVVT', 'KIFEYGFTF', 'GLVECNNHY', 'LLRDKDGVY', 'AFMATNKAY', 'AVLLHEESM', 'LMLADHPEY', 'LSDDAVVCY', 'FTERSDKSY', 'ITAGYNRYY', 'FVSVYFSDY', 'FLFMDRDAL', 'SSLPSYAAY', 'GQWDGWVWL', 'RLFFIDWEY', 'FSVQRNLPF', 'KLQLKGMSY', 'LQIPFAMQM', 'YLAGAGLAF', 'LMQWWSDYV', 'SQKHFDTWW', 'YQHLHTAPK', 'QQYHRFGLY', 'IVFATAARY', 'KMTPWSAYW', 'VVAVGGLAI', 'FVIGGMTGV', 'KQIANELNY', 'LAEQFSGEY', 'NQQVTNSKY', 'NLKLYGAEF', 'RQLESRLGY', 'AQAVMEMTY', 'KIISEIGQL', 'YTAVVPLVY', 'KQFDTYNLW', 'YMIKLAKEV', 'FVRQCFNPM', 'AENGWGFYF', 'KQINPPTVY', 'HTAEIQQFF', 'VQQESSFVM', 'PQVLGGLSF'

In [4]:
def build_classifier(classifier, x_train, y_train, random_state):
    # Choose classifier
    if classifier == "RF":
        clf = RandomForestClassifier(random_state=random_state)
    elif classifier == "SVM":
        clf = svm.SVC(random_state=random_state, probability=True, cache_size = 5000)
    elif classifier == "KNN_3":
        clf = KNeighborsClassifier(n_neighbors=3)
    elif classifier == "KNN_5":
        clf = KNeighborsClassifier(n_neighbors=5)
    elif classifier == "MLP":
        clf = MLPClassifier(random_state=random_state, max_iter=100000)
    elif classifier == "dummy":
        clf = DummyClassifier(random_state=random_state)
    else:
        clf = RandomForestClassifier(random_state=random_state)

    clf.fit(x_train, y_train)

    return clf

def multilayer_perceptrom(x_train, y_train, rand_state):
    clf = MLPClassifier(solver='sgd', activation='tanh', alpha=0.4, hidden_layer_sizes=(82, 31), learning_rate='constant', random_state=rand_state, max_iter=10000)
    clf.fit(x_train, y_train)
    return clf    

In [ ]:
#k fold validation
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
clf = multilayer_perceptrom(X_train, y_train, rand_state=2)
#clf = multilayer_perceptrom(X_train, y_train, 3)
y_pred = clf.predict(X_test)
print("Accuracy k fold: ",accuracy_score(y_test, y_pred))

In [ ]:
df_confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
df_conf_norm = df_confusion / df_confusion.sum(axis=0)

sn.heatmap(df_confusion, cmap="YlOrRd", annot=False)
plt.show()
sn.heatmap(df_conf_norm, cmap="YlOrRd", annot=False)
plt.show()

In [8]:

from scipy.stats import uniform, randint

def hyperparams_svc():
    # parameters to optimize
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    classifier = SVC()
    return classifier, tuned_parameters
    
def hyperparams(classifier, tuned_parameters, X_train, y_train, X_test, y_test):

    scores = ['precision', 'recall']
    #scores = ['recall']

    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(
            classifier, tuned_parameters, scoring='%s_macro' % score
        )
        clf.fit(X_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
        print()

skf = StratifiedKFold(n_splits=10)
for train, test in skf.split(features, labels):
        X_train, X_test = features[train], features[test]
        y_train, y_test = labels[train], labels[test]
                                                                   
classifier, tuned_parameters = hyperparams_svc()
hyperparams(classifier, tuned_parameters, X_train, y_train, X_test, y_test)        


# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}

Grid scores on development set:

0.659 (+/-0.467) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.379 (+/-0.003) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.857 (+/-0.031) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.659 (+/-0.467) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.856 (+/-0.049) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.860 (+/-0.028) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.825 (+/-0.111) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.851 (+/-0.054) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.836 (+/-0.092) for {'C': 1, 'kernel': 'linear'}
0.825 (+/-0.139) for {'C': 10, 'kernel': 'linear'}
0.815 (+/-0.130) for {'C': 100, 'kernel': 'linear'}
0.809 (+/-0.138) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed

In [ ]:
def plotROC(pred_labels, test_labels):
    false_positive_rate, true_positive_rate, thresholds = roc_curve(test_labels, pred_labels)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([-0.1, 1.2])
    plt.ylim([-0.1, 1.2])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    print("ROC-Curve plotted")
    plt.show()
plotROC(y_test, y_pred)    